# Lab 2: Analyzing Water Networks and Energy Use in CA

In this lab, we will be visualizing and analyzing water networks for specific utilities in CA. At this time you should have been assigned a utility to visualize and analyze. Here is a mapping of utilities to their respective codes in the graphs and data.

Utility Code|Utility Name
-|-
1805003E|Alameda County Water District
1803033E|Fresno City of
1810019E|Hi-Desert Water District
1809027E|Palmdale Water District
1805085E|San Jose Water Company
1807341E|Santa Monica City of

<div class="alert alert-info"> 

**QUESTION** cells are in blue and ask you to make graphs, answer conceptual questions, or do other lab tasks. To receive full credit for your lab, you must complete all **QUESTION** cells.

</div>

Note: There is a folder named "ouput" located in the same directory where you opened this lab. Currently it is empty, but later on you will call functions that will ouput to this folder and you will need to copy and paste that ouput to a link. Try to quickly locate that folder in the tab where you opened this lab.

To start off the lab, we have to import some tools for us to be able to select and visualize certain utilities. These tools will build graphs for us which show which water sources and which types of water are upstream from a certain utility. The tools will also enable us to perform energy analysis on the utilities.Therefore, just run the cell and this will set up our environment.

In [2]:
!pip install -q ipywidgets
!pip install -q ipysankeywidget

from sub_network import subWESTnet
import pandas as pd
import numpy as np
import json, urllib, ast
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Here, we create what is called an instance of a graph object. This is what will enable us to be able to visualize the water network associated with a certain utility, all the sources and the types of water that are leading into it. We enter in the number 2010, because we will be looking at the your utility network for that year.

In [4]:
new_upstream_example = subWESTnet('data/fixed_times_erl.csv', 2010)

This line of code specifies that we want to look at all the sources upstream from the utility. The point of this is that we want to analyze what are the types of water coming into these utilities, how much is coming in, and from where is it coming. In the spot that has "Your Utility Name Here" put your utility name in the quotes. To see what your specific utility name is, look at the table at the top of this page and put that name specifically in the quotes.

In [5]:
new_upstream_example.upstream("Your Utility Name Here")

Unweighted upstream graph built.


This line of code is here to compute all the amounts of water going into the different nodes in the network, all you have to do is run the following cell

In [6]:
new_upstream_example.balance_graph()

Graph is weighted.


The following attributes of our upstream example show the tables of the actual network (by using source and target labels) and the energies associated with each node in the graph. Just browse the tables and try to see what each column is referring to and how the rows relate certain utilities/sources with energy levels and the amount of water moving through each point in the network.

In [7]:
new_upstream_example.table()

,source,target,cumulative_volume_af,transmission_kwh/af,treatment_kwh/af,used_vol_af
0,1805089PD,1805085PD,5.975593e+04,25.000,0.00,58952.53
1,SW1804SWP07,SW1805SWP08,6.670851e+04,965.235,0.00,0.00
2,SWP_SFBayDelta,SW1804SWP06,2.485058e+06,0.000,0.00,0.00
3,SFBayDelta,SWP_SFBayDelta,2.521429e+06,0.000,0.00,0.00
4,SW1804SWP06,SW1804SWP07,1.069150e+05,338.920,0.00,0.00
5,SW1805SWP08,1805089PD,4.312799e+04,25.000,88.00,29000.00
6,SW1805081SYS,SW1805081EB,1.146900e+04,0.700,0.00,0.00
7,R_Tuolumn,RES1805081,1.000000e+04,0.000,0.00,0.00
8,RES1805081,SW1805081SYS,7.687000e+03,0.000,0.00,0.00
9,SW1805081EB,1805089PD,4.295548e+04,0.000,88.00,28884.00


In [8]:
new_upstream_example.energy

,node,kwh/af
0,WWT1805083,0.000000
1,R_Tuolumn,0.000000
2,RES1805081,0.000000
3,Lk_OrovilleSWP,0.000000
4,SW1802SWP01,0.000000
5,R_AmericanMFRK,0.000000
6,R_AmericanSFRK,0.000000
7,LK_TrinityCVP,0.000000
8,LK_ShastaCVP,0.000000
9,R_Stanislaus,0.000000


This folowing cell takes the upstream table, and turns it into a dot file. The dotfile is placed in 

ouput/sub_(your_utility_name)_2010__upstream/downstream.dot This folder is on the same level as where this lab is located

There are some things to note about the nodes in the graph. The ones that start with "R_" are a river, ones that contain "SWP" or "CVP" are parts of federal government water management projects in CA, ones with "LK" are lakes, ones with "RES" are reservoirs, and finally the nodes which contain your utility's code and then end with PD, NPD, and GW are potable, non-potable, and ground water types flowing into your utility, repectively.

In [9]:
new_upstream_example.to_dot()

dot saved


Go to the dot file (inside the ouput directory where the lab2 notebook is located) and paste the code in the dot file [here](https://dreampuf.github.io/GraphvizOnline/). Make sure to replace the entire code block that is already in the link with the entire code block in our output.dot files.

<div class="alert alert-info"> 

**QUESTION** What are some of the sources connected to your utility (which whould be located at the bottom of the graph)?. What are the types of water flowing in to your utility? **QUESTION** 

</div>

Your Answer Here (Double click the cell to replace the text)

The next cell is quite complicated, don't bother with trying to understand what it is doing. All it does is try to format our network so that we can use something called a sankey diagram. This diagram will allow us to visualize the water network associated with our utility in a way which shows how much water is actually travelling between utilities and sources and other things like reservoirs, rivers, and aqueducts.

In [10]:
def get_sankey_file(data_path, year, utility, upstream=True):
    gi = subWESTnet(data_path, year)
    gi.upstream(utility)
    
    sank_dicts = []
    for v in gi.edges.values():
        sank_dict = {}
        sank_dict['source'] = v['source']
        sank_dict['target'] = v['target']
        sank_dict['value'] = v['used_vol_af']
        sank_dict['color'] = 'steelblue'
        if v['used_vol_af'] == 0:
            sank_dict['color'] = 'goldenrod'
            sank_dict['value'] = .00001
        sank_dicts.append(sank_dict)
    
    df = pd.DataFrame(sank_dicts)
    df = df[df['color'] != 'goldenrod']
    data = pd.read_csv(data_path)
    
    nodes = []
    for i in df['source'].unique():
        sdict = {}
    
        # check if resource/end or not
        check_resource = data[data['target'] == i].shape
        if check_resource[0] == 0:
            sdict['is resource'] = True
        else:
            sdict['is resource'] = False
        if i[-1:] == 'E':
            sdict['is end'] = True
        else:
            sdict['is end'] = False
    
        # compute in/out volumes
        outv = df[df['source'] == i]['value'].sum()
        inv = df[df['target'] == i]['value'].sum()
    
        # get values to compensate for missing volumes
        extra_case = {}
        if outv < inv:
            if not sdict['is end']:
                extra_case['value'] = inv - outv
                extra_case['source'] = i
                extra_case['target'] = i + " to Other"
                nodes.append(extra_case)
        elif outv > inv:
            if not sdict['is resource']:
                extra_case['value'] = outv - inv
                extra_case['source'] = "Other to " + i
                extra_case['target'] = i
                nodes.append(extra_case)
    
    extra_ = df.drop('color', axis=1).append(pd.DataFrame(nodes)).reset_index(drop=True)
    a = []
    count = 0
    break_count = 0
    break_break = []
    while 'break' != break_break:
        drop = []
        for i in extra_['target']:
            if extra_[extra_['source'] == i].shape[0] == 0:
                if i[-5:] != 'Other' and i[-1:] != 'E':
                    drop.append(i)
        extra_ = extra_.loc[[k for k, v in extra_.iterrows() if v['target'] not in drop]]
        count += len(drop)
        a.extend(drop)
        if count == len(a):
            break_count += 1
        if break_count == 50:
            break_break = 'break'
    
    return extra_


Run the following cell but make sure to put your utility's code, not its name, in the quotes that say "Your Utility Code Here". Remember what all the different codes mean from the previous graph we made. These encodings still apply to the Sankey graph we are going to make in the following cells, except this time your utility will be on the right hand side of the sankey diagram encoded by its utility code, not it's actual name. The graph below represents the flow of water involved with this utility in the year 2010.

In [11]:
SankeyWidget(layout=Layout(width="1500", height="1000"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=get_sankey_file('data/links_erl.csv', 2010, 'Your Utility Code Here', upstream=True).to_dict('records'))

Unweighted upstream graph built.


SankeyWidget(layout=Layout(height='1000', width='1500'), links=[{'source': '1805089PD', 'target': '1805085PD',…

<div class="alert alert-info"> 

**QUESTION** Examine the above graph. Your utility should be on the right hand side and the sources that have water flowing into it to the left of it. You can run your mouse over the graph and see how much water is flowing in each connection. How much water of each type is flowing directly into your utility? **QUESTION**

</div>

Your answer here.

The next graph we are making is the predicted flow of water associated with your utility in 2015. These predictions were made in 2010. Make sure to fill in the "Your Utility Code Here" portion of the cell, so that it can be run, and use your utiliy's code just like in the last Sankey Diagram you made.

In [12]:
SankeyWidget(layout=Layout(width="1500", height="1000"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=get_sankey_file('data/links_erl.csv', 2015, 'Your Utility Code Here', upstream=True).to_dict('records'))

Unweighted upstream graph built.


SankeyWidget(layout=Layout(height='1000', width='1500'), links=[{'source': '1805089PD', 'target': '1805085PD',…

Finally the last graph is the actual 2015 flow of water associated with your utility. We are going to zoom into just the portion that contains your utility and the types of water going into it. Make sure to fill in the "Your utility code here" part of the cell.

In [14]:
SankeyWidget(layout=Layout(width="1000", height="500"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=get_sankey_file('data/links_2015module.csv', 2015, 'Your Utility Code Here', upstream=True).to_dict('records'))

Unweighted upstream graph built.


SankeyWidget(layout=Layout(height='500', width='1000'), links=[{'source': '1805085PD', 'target': '1805085E', '…

<div class="alert alert-info"> 

**QUESTION** Examine the two graphs above and look at the direct connections to your utility. How much water of each type seems to be flowing into your utility in both the predicted 2015 data and the actual 2015 data? How do these levels of water flow compare to 2010 and how do they compare to each other. Do the tyes of water flowing into your uility change at all. Given the change in the flow of water between 2010 and 2015 and how 2015 compares with what was predicted are there any recent events in California which might explain these differences?  **QUESTION**

</div>

Your answers here.